### ПОДКЛЮЧЕНИЕ ИСПОЛЬЗУЕМЫХ БИБЛИОТЕК

In [25]:
import sys

from PyQt5 import uic
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QMessageBox, qApp

from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

import matplotlib.pyplot as plt
import numpy as np
import numexpr as ne

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from stl import mesh
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

# РОДИТЕЛЬСКИЙ КЛАСС

In [26]:
class Window(QMainWindow):
    EXIT_CODE_REBOOT = -123
    def __init__(self, ui):
        super(Window,self).__init__()
        self.ui = uic.loadUi(ui, self)
        self.click()
    
    def SetWidgets(self):
        pass
    
    def click(self):
        pass
    
    def CreateWindowGraph(self):
        pass
    
    def PrintFormula(self):
        pass
    
    def PrintInput(self):
        pass
    
    def PrintFile(self):
        pass
    
    def Reset(self):
        qApp.exit( Window.EXIT_CODE_REBOOT )

#### ГЛАВНЫЙ ИНТЕРФЕЙС

In [27]:
class MainWindow(Window):
    def __init__(self):
        Window.__init__(self,ui = 'mainwindow.ui')
        
        self.inputline_2.setVisible(False)
        self.lineEdit_2.setVisible(False)
        
    def click(self):
        self.exit.clicked.connect(lambda: self.close())
        self.cb_i.stateChanged.connect(lambda: self.ChangeInput())
        self.cb_w.stateChanged.connect(lambda: self.ChangeInputWay())
        
        #self.b_d.clicked.connect(lambda: print(1))
        self.b_f.clicked.connect(lambda: self.CreateWindowGraph())
        #self.b_e.clicked.connect(lambda: print(3))
        
        self.reset.clicked.connect(lambda: self.Reset())
        
    def CheckBox(self):
        if self.cb_w.isChecked() == True and self.cb_i.isChecked() == True:
            QMessageBox.critical(self, "Неправильный ввод", "У вас выбрано 2 или более 'checkbox'", QMessageBox.Ok)
            return False
        elif self.cb_f.isChecked() == True and self.cb_i.isChecked() == True:
            QMessageBox.critical(self, "Неправильный ввод", "У вас выбрано 2 или более 'checkbox'", QMessageBox.Ok)
            return False
        elif self.cb_f.isChecked() == True and self.cb_w.isChecked() == True:
            QMessageBox.critical(self, "Неправильный ввод", "У вас выбрано 2 или более 'checkbox'", QMessageBox.Ok)
            return False
        else: 
            return True
        
    def ChangeInputWay(self):
        if self.CheckBox() == True:
            if self.cb_w.isChecked() == True:
                self.lineEdit.setText('Путь : ')
            else:
                self.lineEdit.setText('f(x) = ')
        else: 
            self.cb_w.setChecked(False)
            print(self.cb_w.isChecked())
        
    def ChangeInput(self):
        if self.CheckBox() == True:
            if self.cb_i.isChecked() == True:
                self.inputline_2.setVisible(True)
                self.lineEdit_2.setVisible(True)
                self.lineEdit.setText('Y = ')
            else: 
                self.inputline_2.setVisible(False)
                self.lineEdit_2.setVisible(False)
                self.lineEdit.setText('f(x) = ')
        else:
            self.cb_i.setChecked(False)
            print(self.cb_i.isChecked())
        
    def CreateWindowGraph(self):
        if self.CheckBox() == True:
            if self.cb_f.isChecked() == True:
                self.w = WindowGraph(['formula',self.inputline.text()])
                self.w.show()
            elif self.cb_i.isChecked() == True:
                self.w = WindowGraph(['input',[self.inputline_2.text(),self.inputline.text()]])
                self.w.show()
            elif self.cb_w.isChecked() == True:
                self.w = WindowGraph(['file',self.inputline.text()])
                self.w.show()
        
    def Print(self):
        data = np.arange(-10,10,0.1)
        self.ax1.clear()
        self.ax1.plot(data, np.sin(data))
        self.canvas.draw()
        
    def pie(self):
        sizes = [17, 15, 11, 15, 14]
        labels = []
        explode = []
        for i in range(len(sizes)):
            string = 'Б03-91' + str(i+1)
            labels.append(string)
            explode.append(sizes[i] / 100)

        self.ax1.clear()
        c = np.random.rand(5, 4)
        self.ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=20, colors = c)
        self.ax1.axis('equal')
        self.ax1.legend(title = 'Номер группы')
        self.canvas.draw()

#### ИНТЕРФЕЙС ГРАФИКОВ(с формулы/входных данных/с файла)

In [31]:
class WindowGraph(Window):
    def __init__(self, argv):
        inputType = argv[0]
        Window.__init__(self,'graph.ui')
        self.SetWidgets()
        
        if (inputType == 'formula'):
            formula = argv[1]
            self.formula.setText(formula)
            self.SetWidgetFalse()
            
        elif (inputType == 'input'):
            args = argv[1]
            ar = []
            s = args[0].split()
            num_list = []
 
            for word in s:
                num_list.append(ne.evaluate(word))
            ar.append(num_list)
            
            s = args[1].split()
            num_list = []
            for word in s:
                num_list.append(ne.evaluate(word))
            ar.append(num_list)
            
            self.PrintInput(ar)
        
        elif (inputType == 'file'):
            self.formula.setText(argv[1])
            self.PrintFile(argv[1])
        
        
    def SetWidgetFalse(self):
        self.save_res.setVisible(False)
        self.w1.setVisible(False)
        self.back.setVisible(False)
        
        
        self.minL.setVisible(True)
        self.maxL.setVisible(True)
        self.step.setVisible(True)
        self.show_g.setVisible(True)
        
        self.minX.setVisible(True)
        self.maxX.setVisible(True)
        self.stepT.setVisible(True)
        
        self.ax.clear()
    
    def SetWidgetTrue(self):
        self.minL.setVisible(False)
        self.maxL.setVisible(False)
        self.step.setVisible(False)
        self.show_g.setVisible(False)
        
        self.minX.setVisible(False)
        self.maxX.setVisible(False)
        self.stepT.setVisible(False)
        
        self.save_res.setVisible(True)
        self.w1.setVisible(True)
        self.back.setVisible(True)
        
    def click(self):
        self.show_g.clicked.connect(lambda: self.PrintFormula())
        self.back.clicked.connect(lambda: self.SetWidgetFalse())
        
    def SetWidgets(self):
        self.figure = plt.Figure()
        self.canvas = FigureCanvas(self.figure)
        self.toolbar = NavigationToolbar(self.canvas, self)
        
        layout = QVBoxLayout()
        layout.addWidget(self.canvas)
        layout.addWidget(self.toolbar)
        self.w1.setLayout(layout)
        self.ax = self.figure.add_subplot(1,1,1)
        self.ax.grid(True)
        
    def PrintFile(self, file):
        self.SetWidgetTrue()
        self.back.setVisible(False)
        
    def PrintInput(self,args):
        self.x, self.y = args[0],args[1]
        self.SetWidgetTrue()
        self.back.setVisible(False)
        self.lineEdit_3.setVisible(False)
        self.formula.setVisible(False)
        try:
            self.ax.plot(self.x, self.y)
        except ValueError:
            QMessageBox.critical(self, "Ошибка ", "Несовпадение размеров 'x' и 'y'", QMessageBox.Ok)
            self.deleteLater()
            return
        
    def PrintFormula(self):
        try:
            x = np.arange(float(self.minL.text()),float(self.maxL.text()),float(self.step.text()))
        except ValueError:
            QMessageBox.critical(self, "Ошибка ", "Неправильные параметры 'x'", QMessageBox.Ok)
            return
        except ZeroDivisionError:
            QMessageBox.critical(self, "Ошибка ", "Неподходящий диапазон 'x'", QMessageBox.Ok)
            return
        
        self.SetWidgetTrue()
        
        y = ne.evaluate(self.formula.text())
        
        self.ax.plot(x,y)
        self.x,self.y = x,y
            
    def SaveRes(self):
        pass

In [32]:
def main():
    currentExitCode = Window.EXIT_CODE_REBOOT
    while currentExitCode == MainWindow.EXIT_CODE_REBOOT:
        app = QApplication(sys.argv)
        window = MainWindow()
        #window = WindowGraph()
        window.show()
        currentExitCode = app.exec_()
        app = None

In [ ]:
if __name__ == '__main__':
     main()